In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# # Table of Contents


1. [Importing Libraies](#1)
 
2.  [Extracting Files](#2)

     2.1 [ Setting Path](#2.1)
     
3.  [Making Data Frame](#3)

4.  [Data Distribution](#4)

      4.1 [Displaying Images of Cat](#4.1)
      
      4.2  [Displaying Images of Dog](#4.2)
      
5. [Siplitting The Data](#5)

    5.1  [Image Data Generator](#5.1)
    
6. [Making CNN Model](#6)

    6.1  [Displaying the model](#6.1)


7 [Training of Model Model](#7)




     


# **Classification Basic CNN.**

Please Vote Up if you like this Note Book

In this notebook, we will learn how to classify images of dog and cat by building a simple Neural Network.

What we will learn:

* Load the images.

* Visulaise the Data distribution of all data.

* Visualizing some of the images
 
* Making The CNN Network

* Use Image Data Generator

* Graph the training loss and validation loss

* Predict the results

* Confusion Matrix

<a id="1"></a> <br>
# 1. Importing Libraries

We are Importing Libraries.

* Libraries which need for

* Image Processing

* Data visualization

*  Making Model Architecture.



In [ ]:
''' importing library''' 

import numpy as np
import sys
import tensorflow as tf
import os
import sys
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, BatchNormalization


%matplotlib inline
import matplotlib.image as img
import matplotlib.pyplot as plt

**Setting Seed**

In [ ]:
'''setting seed'''
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

<a id="2"></a> <br>
# 2. Extarcting Files

In [ ]:
import zipfile

zip_files = ['test1', 'train']

for zip_file in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

In [ ]:
'''test1, train Data is in current working folder'''
print(os.listdir('../working'))

<a id="2.1"></a> <br>
# 2.1 Setting Path

* Setting the path and Image size 

In [ ]:
IMAGE_FOLDER_PATH = "../working/train"
FILE_NAMES = os.listdir(IMAGE_FOLDER_PATH)
WIDTH = 150
HEIGHT = 150

In [ ]:
FILE_NAMES[0:5]

In [ ]:
labels = []
for i in os.listdir(IMAGE_FOLDER_PATH):
    labels+=[i]

<a id="3"></a> <br>
# 3 Making Data Frame

In [ ]:
# empty list
targets = list()
full_paths = list()
train_cats_dir = list()
train_dogs_dir = list()

# finding each file's target
for file_name in FILE_NAMES:
    target = file_name.split(".")[0] # target name
    full_path = os.path.join(IMAGE_FOLDER_PATH, file_name)
    
    if(target == "dog"):
        train_dogs_dir.append(full_path)
    if(target == "cat"):
        train_cats_dir.append(full_path)
    
    full_paths.append(full_path)
    targets.append(target)

dataset = pd.DataFrame() # make dataframe
dataset['image_path'] = full_paths # file path
dataset['target'] = targets # file's target

In [ ]:
dataset.head(10)

In [ ]:
print("total data counts:", dataset['target'].count())
counts = dataset['target'].value_counts()
print(counts)

<a id="4"></a> <br>
# 4 Data Distributation

In [ ]:
label_count=dataset['target'].value_counts()
explode = (0.01,0.01)  

fig1, ax1 = plt.subplots(figsize=(5,5))
ax1.pie(label_count.values, explode=explode, labels=label_count.index, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 
plt.title('Target Distribution')
plt.show()

<a id="4.1"></a> <br>
# 4.1 Displaying Images of Cat

In [ ]:
rows = 4
cols = 4
axes = []
fig=plt.figure(figsize=(10,10))
i = 0

for a in range(rows*cols):
    b = img.imread(train_cats_dir[i])
    axes.append(fig.add_subplot(rows,cols,a+1))
    plt.imshow(b)
    i+=1
fig.tight_layout()
plt.show()

<a id="4.2"></a> <br>
# 4.2 Displaying Images of Dog

In [ ]:
rows = 4
cols = 4
axes = []
fig=plt.figure(figsize=(10,10))
i = 0

for a in range(rows*cols):
    b = img.imread(train_dogs_dir[i])
    axes.append(fig.add_subplot(rows,cols,a+1))
    plt.imshow(b)
    i+=1
fig.tight_layout()
plt.show()

<a id="5"></a> <br>
# 5 Splitting The data

In [ ]:
dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=seed)

# **Data Distributaion Visulatization**

In [ ]:
class_id_distributionTrain = dataset_train['target'].value_counts()
class_id_distributionTrain.head(10)

In [ ]:
plt.figure(figsize=(13,5))
plt.xticks(np.arange(43))
plt.bar(class_id_distributionTrain.index, class_id_distributionTrain.values)

In [ ]:
label_count=dataset_train['target'].value_counts()
explode = (0.01,0.01)  

fig1, ax1 = plt.subplots(figsize=(5,5))
ax1.pie(label_count.values, explode=explode, labels=label_count.index, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 
plt.title('Target Distribution')
plt.show()

In [ ]:
class_id_distributionTest = dataset_test['target'].value_counts()
class_id_distributionTest.head(10)

In [ ]:
label_count=dataset_test['target'].value_counts()
explode = (0.01,0.01)  

fig1, ax1 = plt.subplots(figsize=(5,5))
ax1.pie(label_count.values, explode=explode, labels=label_count.index, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 
plt.title('Target Distribution')
plt.show()

<a id="5.1"></a> <br>
# 5.1 Image DataGenerator

When there is little data to train, we have to use ImageDataGenerator to increase the number of data.
It is recommended to use only scaling for test data.

1. rescale = 1./255 : change the value between 0 and 1

2. rotation_range = 15 : Random rotation within 15 degrees

3. shear_range = 0.1 : shear range 10%

4. zoom_range = 0.2 : zoom range 20%

5. horizontal_flip = True : Randomly flip horizontally.

6. width_shift_range = 0.1 : Randomly move the original image horizontally within 10% of the width

7. height_shift_range=0.1 : Randomly move the original image vertically within 10% of the width


In [ ]:
train_datagen=ImageDataGenerator(
rotation_range=15,
rescale=1./255,
shear_range=0.1,
zoom_range=0.2,
horizontal_flip=True,
width_shift_range=0.1,
height_shift_range=0.1)

train_datagenerator=train_datagen.flow_from_dataframe(dataframe=dataset_train,
                                                     x_col="image_path",
                                                     y_col="target",
                                                     target_size=(WIDTH, HEIGHT),
                                                     class_mode="binary",
                                                     batch_size=150)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_datagenerator=test_datagen.flow_from_dataframe(dataframe=dataset_test,
                                                   x_col="image_path",
                                                   y_col="target",
                                                   target_size=(WIDTH, HEIGHT),
                                                   class_mode="binary",
                                                   batch_size=150)

<a id="6"></a> <br>
# 6 Making CNN Model

In [ ]:
from tensorflow.keras.layers import Dense, BatchNormalization
model = Sequential() # implement model layer 
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(WIDTH, HEIGHT, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(MaxPooling2D(pool_size=2))
model.add(Dense(1280, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))
#model.add(Dropout(0.5))
#model.add(Dense(1, activation='sigmoid'))
model.add(Flatten())

<a id="6.1"></a> <br>
# 6.1 Displaying Model

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png') 

In [ ]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

<a id="7"></a> <br>
# 7 Train The model

In [ ]:
History=model.fit(train_datagenerator,
                       epochs=1,
                       validation_data=test_datagenerator,
                       validation_steps=dataset_test.shape[0]/150,
                       steps_per_epoch=dataset_train.shape[0]/150)

<a id="8"></a> <br>
# 8 Plotting The Result

In [ ]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']
loss = History.history['loss']
val_loss = History.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

[](http://)



<a id="9"></a> <br>
# 9 Predict the Results

In [ ]:
 test_loss, test_acc = model.evaluate(test_datagenerator, steps=len(test_datagenerator), verbose=1)
 print('Loss: %.3f' % (test_loss * 100.0))
 print('Accuracy: %.3f' % (test_acc * 100.0)) 

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
predictions = model.predict(x=test_datagenerator, steps= len(test_datagenerator), verbose=0)

In [ ]:
test_datagenerator.classes

In [ ]:
cm = confusion_matrix(y_true=test_datagenerator.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black")
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')




<a id="10"></a> <br>
# 10 onfusion Matrix Plot


In [ ]:
cm_plot_labels = ['0_cat', '1_dog']

plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')